In [1]:
# FOR HDI: http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Information.php
# Human development classification
# HDI classifications are based on HDI fixed cutoff points, which are derived from the quartiles of dis
# tributions of the component indicators. The cutoffpoints are HDI of less than 0.550 for low human development,
#  0.550–0.699 for medium human development, 0.700–0.799 for high human development and 0.800 or greater 
#  for very high human development.

In [2]:
import json
from turtle import back
import requests

In [3]:
url_HDRO = "http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id=137506,44206/year=2019"
url_USGS = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"

response_HDRO = requests.get(url_HDRO)
print(f"HDRO: {response_HDRO}")
response_USGS = requests.get(url_USGS)
print(f"USGS: {response_USGS}")

HDRO: <Response [200]>
USGS: <Response [200]>


In [4]:
data_HDRO = response_HDRO.json()
# print(json.dumps(data_HDRO, indent=4, sort_keys=True))
data_USGS = response_USGS.json()

In [5]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import module to create relationship patterns
from sqlalchemy.orm import relationship

# Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Table, Column, Integer, String, Float, BigInteger, ForeignKey

In [6]:
# Create classes:
# Countries & Earthquakes

class Countries(Base):  #Parent
    __tablename__ = 'countries'
    # id = Column(Integer, primary_key=True)
    CountryCode = Column(String(255), primary_key=True)
    CountryName = Column(String(255))
    Population = Column(Float)  # Millions
    HDIndex = Column(Float) # Index
    earthquake = relationship("Earthquakes")

class Earthquakes(Base):    #Child
    __tablename__ = 'earthquakes'
    # id = Column(Integer, primary_key=True)
    EarthquakeID = Column(String(255), primary_key=True)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Depth = Column(Float)
    DateTime = Column(BigInteger)
    # CountryCode = Column(String(255))   # Foreign Key?
    country_id = Column(String(255), ForeignKey('countries.CountryCode'))
    # parent = relationship("Countries", back_populates="children")


# print(data_USGS["features"])

In [7]:
import reverse_geocoder as rg

In [13]:
# Create Database Connection
# ---------------------------
# Establish Connection
engine = create_engine("sqlite:///project2-3.sqlite")
conn = engine.connect()

# Create Countries and Earthquakes tables within the database
Base.metadata.create_all(conn)

# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

i=1
# Create specific instances of the Countries and Earthquakes classes
for earthquake in data_USGS["features"]:

    if i<=10:
        # print(earthquake["id"], earthquake["geometry"]["coordinates"][0], earthquake["geometry"]["coordinates"][1], earthquake["geometry"]["coordinates"][2], earthquake["properties"]["time"])
        lon = earthquake["geometry"]["coordinates"][0]
        lat = earthquake["geometry"]["coordinates"][1]
        depth1 = earthquake["geometry"]["coordinates"][2]
        ctry_code = rg.search((lat,lon))[0]["cc"]
        new_earthquake = Earthquakes(EarthquakeID=earthquake["id"], Longitude=lon, Latitude=lat, Depth=depth1, DateTime=earthquake["properties"]["time"], country_id=ctry_code)

        print(f"{i}, {earthquake['id']}, {lon}, {lat}, {depth1}, {earthquake['properties']['time']}, {ctry_code}")
        # Add new_earthquake to the current session
        session.add(new_earthquake)
        # Add new_country to the current session

        # Commit objects to the database
        session.commit()
        # print(i)
        i = i+1
    else:
        break


conn.close()
engine.dispose()




earthquake_list = session.query(Earthquakes)
for earthquake in earthquake_list:
    print(earthquake.Longitude)

1, nc73679671, -122.8286667, 38.814167, 1.71, 1642425219890, US
2, nc73679666, -122.8040009, 38.8358345, 1.84, 1642424621790, US
3, nc73679661, -122.765831, 38.8243332, 2.21, 1642422795980, US
4, nc73679656, -122.8551636, 38.8250008, 2.2, 1642422566460, US
5, nc73679651, -122.4076691, 38.6626663, 8.86, 1642422297540, US
6, ci39914471, -117.7183333, 35.8935, 8.12, 1642421532580, US
7, ak022sbbg1t, -149.8531, 61.8252, 76.2, 1642421187214, US
8, hv72878212, -155.399169921875, 19.2024993896484, 31.5900001525879, 1642420716640, US
9, ok2022bedg, -99.27513885, 36.51202774, 2.934551954, 1642419786772, US
10, us7000gclu, 63.5686, 34.9479, 10, 1642419607217, AF
-122.8286667
-122.8040009
-122.765831
-122.8551636
-122.4076691
-117.7183333
-149.8531
-155.399169921875
-99.27513885
63.5686
